# Exploratory analysis

Exploratory analysis helps you understand features of your data which are relevant linking or deduplicating your data.

Splink includes a variety of charts to help with this, which are demonstrated in this notebook.


### Read in the data

For the purpose of this tutorial we will use a 1,000 row synthetic dataset that contains duplicates.

The first five rows of this dataset are printed below.

Note that the cluster column represents the 'ground truth' - a column which tells us with which rows refer to the same person. In most real linkage scenarios, we wouldn't have this column (this is what Splink is trying to estimate.)


In [1]:
!pip install --upgrade pip

In [2]:
!pip install 'splink[spark]'

  Using cached splink-4.0.6-py3-none-any.whl.metadata (11 kB)
Using cached splink-4.0.6-py3-none-any.whl (3.7 MB)


In [3]:
import splink, pyspark, duckdb, sqlite3
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("Spark version:", spark.version)
print("Splink version:", splink.__version__)
print("PySpark version:", pyspark.__version__)
print("DuckDB version:", duckdb.__version__) 
print("SQLite3 version:", sqlite3.sqlite_version) 

your 131072x1 screen size is bogus. expect trouble
25/03/02 10:46:14 WARN Utils: Your hostname, dev resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/02 10:46:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 10:46:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.5
Splink version: 4.0.6
PySpark version: 3.5.5
DuckDB version: 1.2.0
SQLite3 version: 3.45.1


In [4]:
from splink import  splink_datasets

df = splink_datasets.fake_1000
# df = df.drop(columns=["cluster"])
df.head(20)

,unique_id,first_name,surname,dob,city,email,cluster
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net,0
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net,0
2,2,Rob,Allen,1971-06-24,London,roberta25@smith.net,0
3,3,Robert,Alen,1971-06-24,Lonon,NaN,0
4,4,Grace,NaN,1997-04-26,Hull,grace.kelly52@jones.com,1
5,5,Grace,Kelly,1991-04-26,NaN,grace.kelly52@jones.com,1
6,6,Logan,pMurphy,1973-08-01,NaN,NaN,2
7,7,NaN,NaN,2015-03-03,Portsmouth,evied56@harris-bailey.net,3
8,8,NaN,Dean,2015-03-03,NaN,NaN,3
9,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net,3


## Analyse missingness


It's important to understand the level of missingness in your data, because columns with higher levels of missingness are less useful for data linking.


In [5]:
from splink.exploratory import completeness_chart
from splink import DuckDBAPI
db_api = DuckDBAPI()
completeness_chart(df, db_api=db_api)

alt.LayerChart(...)

The above summary chart shows that in this dataset, the `email`, `city`, `surname` and `forename` columns contain nulls, but the level of missingness is relatively low (less than 22%).


## Analyse the distribution of values in your data


The distribution of values in your data is important for two main reasons:

1. Columns with higher cardinality (number of distinct values) are usually more useful for data linking. For instance, date of birth is a much stronger linkage variable than gender.

2. The skew of values is important. If you have a `city` column that has 1,000 distinct values, but 75% of them are `London`, this is much less useful for linkage than if the 1,000 values were equally distributed

The `linker.profile_columns()` method creates summary charts to help you understand these aspects of your data.

To profile all columns, leave the column_expressions argument empty.


In [6]:
from splink.exploratory import profile_columns

profile_columns(df, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

This chart is very information-dense, but here are some key takehomes relevant to our linkage:

- There is strong skew in the `city` field with around 20% of the values being `London`. We therefore will probably want to use `term_frequency_adjustments` in our linkage model, so that it can weight a match on London differently to a match on, say, `Norwich`.

- Looking at the "Bottom 5 values by value count", we can see typos in the data in most fields. This tells us this information was possibly entered by hand, or using Optical Character Recognition, giving us an insight into the type of data entry errors we may see.

- Email is a much more uniquely-identifying field than any others, with a maximum value count of 6. It's likely to be a strong linking variable.


!!! note "Further Reading"

For more on exploratory analysis tools in Splink, please refer to the [Exploratory Analysis API documentation](https://moj-analytical-services.github.io/splink/api_docs/exploratory.html).

For more on the charts used in this tutorial, please refer to the [Charts Gallery](https://moj-analytical-services.github.io/splink/charts/index.html#exploratory-analysis).


## Next steps

At this point, we have begun to develop a strong understanding of our data. It's time to move on to estimating a linkage model
